In [5]:
import pandas as pd
import sqlite3
import sqlalchemy
import random
from datetime import timedelta
import numpy as np

In [6]:
reactions = pd.read_csv('sampleReactions.csv', sep="\t")
hashtags = ["#stayhome", "#lockdown", "#lockdown", "#socialdistancing", "#socialdistancing", "#lockdown", "#longcovid", "#freeagain", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#workfromhome", "#workfromhome", "#stayhome", "#stayhome", "#vaccination", "#quarantine", "#flattenthecurve", "#stayhome", "#lockdown", "#longcovid","","","","","","","","","","","","","","","",""]
for (i, row) in enumerate(reactions.iterrows()):
    reactions.loc[i, 'comment'] = row[1]['comment'] + " " +random.choice(hashtags)
    
reactions['comment']


0      while most the country was all about Baby Shar...
1        What happened with Chernobyl? #socialdistancing
2      Surprised there wasn’t another Baby Shark spik...
3      Montana being the only state with Far Cry 5 wa...
4      Fortnite lasted longer than George Floydd, BLM...
                             ...                        
470    I remember these huge ships getting paid to ta...
471    Why does the simulation end up making the patc...
472    Notice the steady stream coming out of China. ...
473    this seems like bullshit... &#x200B; In Januar...
474    What’s brilliant about that is it works like a...
Name: comment, Length: 475, dtype: object

In [7]:
df = pd.read_json("./sampleComments.json")
df['id'] = df.index
df = df.drop(['value'], axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['date'] = df['date'].dt.date
df['enddate'] = df['date']
df['createdAt'] = pd.to_datetime('today').normalize()
df['updatedAt'] = pd.to_datetime('today').normalize()

#df.drop(df[df.country != "Germany"].index, inplace=True)

for (i, row) in enumerate(df.iterrows()):
    reaction = reactions.sample()
    df.loc[i, 'comment'] = reaction['comment'].values[0]
    df.loc[i, 'author'] = reaction['author'].values[0]
    if (i % 3) is 0:
        newDate = row[1].date + timedelta(days=random.randint(5,20))
        df.loc[i, 'enddate'] = newDate


df

,date,comment,weight,country,id,enddate,createdAt,updatedAt,author
0,2021-01-22,"Hello, I'm the mod of /r/TheOceanCleanup which...",4,Fiji,0,2021-01-29,2021-03-12,2021-03-12,houston_wehaveaprblm
1,2020-06-11,This doesnt include if he actually played golf...,4,Lebanon,1,2020-06-11,2021-03-12,2021-03-12,vaguevlogger
2,2020-08-10,"90% were likely by tiktok users, big trend on ...",4,Iceland,2,2020-08-10,2021-03-12,2021-03-12,Jibbers8
3,2020-07-26,I really enjoy this graph. It really bothers m...,4,Lesotho,3,2020-08-01,2021-03-12,2021-03-12,work2305
4,2020-09-08,Is this US or what? Worth putting in the title...,3,Mongolia,4,2020-09-08,2021-03-12,2021-03-12,ModeHopper
...,...,...,...,...,...,...,...,...,...
49995,2020-11-05,"So many people here talking about Dexter, GoT ...",3,El Salvador,49995,2020-11-22,2021-03-12,2021-03-12,JackReact
49996,2020-03-16,Wow that's crazy I didn't think a company coul...,4,Latvia,49996,2020-03-16,2021-03-12,2021-03-12,Not_so_ghetto
49997,2020-06-29,It stayed at exactly 25 percent in the UK for ...,4,Iceland,49997,2020-06-29,2021-03-12,2021-03-12,rcarmack1
49998,2020-10-31,Tfw you realize the blockbuster in your hometo...,4,Luxembourg,49998,2020-11-08,2021-03-12,2021-03-12,Picturesof_Animals


In [8]:
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
df.to_sql(con=sqlite, index=False, name='memories', if_exists='replace')